In [4]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import cv2

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
print(face_cascade)

<CascadeClassifier 0x7fb8d4276c10>


In [6]:
 def face_extract(img):
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped=img[y:y+h, x:x+w]
    
    return cropped


In [7]:
cap=cv2.VideoCapture(0)
count=0
while(True):
    ret, frame=cap.read()
    if face_extract(frame) is not None:
        count+=1
        
        img= cv2.resize(face_extract(frame), (200,200))
        #img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        file_name_path='./Faces/Vidheep/'+str(count)+'.jpg'
        cv2.imwrite(file_name_path, img)
        
        cv2.putText(img, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0),2)
        cv2.imshow('image',img)
        if cv2.waitKey(13)& 0xFF == ord('q'):
            break
        
    if count==1000:
        break
        
cap.release()
cv2.destroyAllWindows()

In [12]:
train=[]
label=[]
names=[ 'Kashish', 'Rakshit', 'Aditya']
for name in names:
    for i in range(1,100):       
        photo=cv2.imread('./Faces/'+name+'/'+ str(i)+'.jpg')
        train.append(np.asarray(photo))
        label.append(name)
 

In [13]:
data=pd.DataFrame({'input':train, 'label':label})
data['label']=label
df = data.sample(frac=1).reset_index(drop=True)

In [14]:
y=np.array(df['label'])
y.shape

(297,)

In [15]:
z=[]
for i in range(df.shape[0]):
    z.append(df['input'][i])
z=np.array(z)
z.shape
#z.reshape(267,200,200)
#print(z)

(297, 200, 200, 3)

In [16]:
print(z.shape)

(297, 200, 200, 3)


In [17]:
X_train= z[:230]
X_test=z[230:]
y_train= y[:230]
y_test=y[230:]

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(230, 200, 200, 3)
(67, 200, 200, 3)
(230,)
(67,)


# Model Building

In [19]:
import keras
import time

Using TensorFlow backend.


In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.callbacks import TensorBoard

In [21]:
name="face-cnn-{}".format(int(time.time()))
print(name)

face-cnn-1561731120


In [22]:
names=['Kashish', 'Aditya', 'Rakshit']

In [23]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(names)
print(integer_encoded)

[1 0 2]


In [24]:
y_tr=label_encoder.transform(y_train)
y_te=label_encoder.transform(y_test)
print(y_tr)

[0 2 0 2 1 2 0 1 1 1 0 1 0 1 1 1 2 1 1 2 1 0 0 1 1 1 0 2 1 2 1 0 0 1 0 1 1
 1 0 0 2 0 2 0 2 1 0 2 0 0 2 0 1 2 1 0 2 2 2 2 2 2 0 2 2 2 1 1 1 1 1 1 1 2
 2 0 1 2 2 1 0 0 1 0 1 2 1 2 2 1 2 0 1 1 1 2 0 2 1 2 2 1 0 2 2 1 0 2 2 1 2
 2 1 1 1 0 2 0 2 0 2 1 0 0 2 2 1 1 2 1 1 0 0 1 0 0 2 0 2 0 1 0 2 1 2 2 2 2
 0 2 0 0 2 1 0 2 2 1 1 0 2 0 2 2 0 1 2 0 2 0 1 0 2 0 1 2 2 0 0 0 2 2 0 2 1
 0 2 2 1 0 2 1 1 1 2 0 1 1 0 1 1 1 0 0 2 2 0 0 1 2 1 2 2 1 1 0 1 0 2 0 1 1
 0 1 2 1 0 0 1 0]


In [25]:
from keras.utils import to_categorical
y_binary = to_categorical(y_tr)
print(y_binary)

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1.

In [ ]:
model=Sequential()
model.add(Conv2D(64, (3,3), input_shape=(200,200,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(X_train, y_binary, batch_size=32, epochs=10, validation_split=0.1)

W0628 19:42:04.559697 140432758654784 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 19:42:04.616081 140432758654784 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 19:42:04.620590 140432758654784 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 19:42:04.660490 140432758654784 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0628 19:42:04.

Train on 207 samples, validate on 23 samples
Epoch 1/10
207/207 [==============================] - 15s 73ms/step - loss: 9.8284 - acc: 0.3333 - val_loss: 12.6142 - val_acc: 0.2174
Epoch 2/10
207/207 [==============================] - 12s 59ms/step - loss: 10.3561 - acc: 0.3575 - val_loss: 12.6142 - val_acc: 0.2174
Epoch 3/10
207/207 [==============================] - 12s 60ms/step - loss: 10.3561 - acc: 0.3575 - val_loss: 12.6142 - val_acc: 0.2174
Epoch 4/10
207/207 [==============================] - 15s 71ms/step - loss: 10.3561 - acc: 0.3575 - val_loss: 12.6142 - val_acc: 0.2174
Epoch 5/10
207/207 [==============================] - 15s 72ms/step - loss: 10.3561 - acc: 0.3575 - val_loss: 12.6142 - val_acc: 0.2174
Epoch 6/10


In [ ]:
dense_layers=[0,1,2]
layer_sizes= [32, 64, 128]
conv_layers=[1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name= "{}- conv-{}-nodes- {}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print("********************* "+name+" ************************")  
            
            model=Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:], activation='relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for i in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            for i in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
            
            model.add(Dense(3, activation='softmax')) 
            tensorboard = TensorBoard(log_dir="logs/{}".format(name))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
            model.fit(X_train, y_binary, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])
            